In [41]:
import numpy as np
import matplotlib.pyplot as plt
import tempfile

def lake_poll(T, start_x, start_y, con):
    Lx, Ly = 10.0, 10.0  # 湖的大小
    Nx, Ny = 100, 100  #分成100格
    dx, dy = Lx / (Nx - 1), Ly / (Ny - 1)  # 間距
    alpha = 0.1  # 擴散係數
    dt = 0.01  
    T = T  # 經歷時常
    nt = int(T / dt)  # 總共跑幾次
    u = np.zeros((Nx, Ny))#預設一開始湖的污染物濃度都是0
    start_x, start_y = int(start_x), int(start_y)  # 污染物投放在哪個點
    u[start_x, start_y] = con  # 污染物的濃度
    
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 5), sharex=True, sharey=True)
    #畫一個(2,5)總共有十張子圖的圖，共用ｘ和 y軸
    Y, X = np.meshgrid(np.linspace(0, Ly, Ny), np.linspace(0, Lx, Nx)) 
    
    # 定義有限差法法
    def diffusion_step(u, alpha, dx, dy, dt):
        u_new = u.copy()
        for i in range(1, Nx-1):
            for j in range(1, Ny-1):
                u_new[i, j] = u[i, j] + alpha * dt * (
                    (u[i+1, j] - 2*u[i, j] + u[i-1, j]) / dx**2 +
                    (u[i, j+1] - 2*u[i, j] + u[i, j-1]) / dy**2
                )
        return u_new

    # 畫第一張子圖，是放下污染物的第0秒
    ax = axes[0, 0]
    im = ax.pcolormesh(X, Y, u, shading='auto', cmap='viridis')
    plt.colorbar(im, ax=ax)
    ax.set_title('Time: 0')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    # 隨時間進行擴散
    pltint = nt // 8 #把時間步數分八塊
    pltc = 1 #拿來計算圖的位置
    for n in range(nt):
        u = diffusion_step(u, alpha, dx, dy, dt)
        if (n + 1) % pltint == 0 and pltc < 10:
            current_time = (n+1) * dt
            row = (pltc // 5)
            col = (pltc % 5)
            ax = axes[row, col]
            im = ax.pcolormesh(X, Y, u, shading='auto', cmap='viridis')
            plt.colorbar(im, ax=ax)
            ax.set_title(f'Time: {current_time:.2f}')
            if col == 0:
                ax.set_ylabel('Y')
            if row == 1:
                ax.set_xlabel('X')
            pltc += 1
            #只有每排的第一張圖有label
    # Final state
    if nt%8 !=0 : #如果上面畫八張圖剛好有包括最後狀態，就不用畫第十張圖
        row = 1
        col = 4
        ax = axes[row, col]
        im = ax.pcolormesh(X, Y, u, shading='auto', cmap='viridis')
        plt.colorbar(im, ax=ax)
        ax.set_title(f'Time: {T:.2f}')
        
        

    # 儲存暫時圖片，用介面顯示出來
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmpfile:
        plt.savefig(tmpfile.name)
        final_image_file = tmpfile.name
    plt.close()

    return u[start_x, start_y], final_image_file

In [52]:
import flet as ft
import nest_asyncio

def main(page: ft.Page):
    global start_x, start_y, start_concentration, reaction_time, result_text, img

    # gui 的介面
    page.title = "pollutant diffusion problem"
    page.window_width = 1000
    page.window_height = 600
    page.horizontal_alignment = ft.CrossAxisAlignment.CENTER

    #圖片的大小
    img = ft.Image(width=1400, height=300)
    
    def result_click(e):#定義result這個button
        x = int(start_x.value)
        y = int(start_y.value)
        con = int(start_concentration.value)
        t = int(reaction_time.value)
        result, final_image_file = lake_poll(t, x, y, con)#帶入上面的函式
        result_text.value = f"Concentration at the starting point after {t} seconds: {result}"
        #輸出結果文字
        img.src = final_image_file
        page.update()
        #更新圖片
        
        
    start_x = ft.Slider(min=0, max=100, width=300, value=50, divisions=99)
    start_x_value = ft.Text(value=f"Start point X coordinate: {(start_x.value)/10:.1f}")
    def update_x(e):
        start_x_value.value = f"Start point X coordinate: {(start_x.value)/10:.1f}"
        page.update()
    
    start_x.on_change = update_x
    #調整x座標的滑桿，並隨時更新移到多少
    start_y = ft.Slider(min=0, max=100, width=300, value=50, divisions=99)
    start_y_value = ft.Text(value=f"Start point Y coordinate: {(start_y.value)/10:.1f}")
    
    def update_y(e):
        start_y_value.value = f"Start point Y coordinate: {(start_y.value)/10:.1f}"
        page.update()
    
    start_y.on_change = update_y
    #調整y座標的滑桿，並隨時更新移到多少
    start_concentration = ft.TextField(label="Initial concentration", width=300, text_size=18)
    reaction_time = ft.TextField(label="Reaction time(s)", width=300, text_size=18)
    result_button = ft.ElevatedButton(text="Show result", width=300, on_click=result_click)
    result_text = ft.Text("", size=18)

    #排好顯示頁面的樣子
    row1 = ft.Row([start_x, start_x_value], alignment=ft.MainAxisAlignment.CENTER)
    row2 = ft.Row([start_y, start_y_value], alignment=ft.MainAxisAlignment.CENTER)
    row3 = ft.Row([start_concentration, reaction_time], alignment=ft.MainAxisAlignment.CENTER)
    #放進介面
    page.add(
            row1,
            row2,
            row3,
            result_button,
            result_text,
            img
            )

nest_asyncio.apply()
ft.app(target=main)